# Vehicle Routing Problem with Time Windows

The vehicle routing problem with time windows (VRPTW) is a combinatorial optimization problem in the area of logistics that aims to find a cheapest set of routes for a given fleet of vehicles such that all customers are served within their time windows and the vehicle capacity is never exceeded.

The problem is formally defined as follows.
We are given
* a depot 0 where all vehicles are located in the beginning,
* a set of customers $C = \{1,\ldots,N\}$ (we define locations $L = \{0\} \cup C$),
* a demand $d_i \ge 0$ for each customer $i \in C$ (the depot has no demand, i.e., $d_0 = 0$),
* a time window $[a_i,b_i]$ with $0 \le a_i \le b_i$ for each location $i \in L$ (including the depot),
* a service time $s_i \ge 0$ for each customer $i \in C$ (the depot has zero service time, i.e., $s_0 = 0$),
* costs $c_{ij} \ge 0$ and time $t_{ij} \ge 0$ for traveling from location $i$ to location $j$,
* and a homogeneous fleet $K=\{1,\ldots,|K|\}$ of vehicles with load capacity $Q > 0$.

We want to find
* a route for each vehicle, i.e., an ordered sequence of customers
* with minimal total routing costs,
such that
* each customer is visited exactly once,
* the start times of the services have to be within the depot and customer time windows (potentially introducing waiting times between services),
* and the vehicle capacity is never exceeded.

This notebook demonstrates how to model the VRPTW as compact MILP, without use of sophisticated column generation or cutting plane methods as needed in state-of-the-art approaches. Therefore, the presented models have limited performance compared to the state-of-the-art, independent of the used solver.

## Instance Configuration

The first part is dedicated to setting up the instance data. For demonstration purposes, we generate artificial data in the following way:
* We predefine the number of customers, the number of available vehicles, their load capacity, a time horizon (at which all customer services have to be finished), and a range for the customer demands, time window widths, and service times.
* We place the depot and customers randomly in a 1000x1000 grid. This allows us to compute Euclidian distances which lead to quite realistic relations between different connections. These distances are interpreted as traveling costs. Travel times are based on the costs but scaled to the time horizon.
* Depot and customer demands, time windows, and service times are randomly generated in their predefined ranges.

In [ ]:
import math
import random

### INSTANCE CONFIGURATION
# Example 1 (wide time windows):   30, 5, 30, 2-5, 100, 80-90, 2-4
# Example 2 (narrow time windows): 30, 5, 30, 2-5, 100, 10-15, 2-4
numCustomers = 30
maxNumVehicles = 5

vehicleCapacity = 30
demandRange = (2, 5)

timeHorizon = 100
timeWindowWidthRange = (10, 15)
serviceTimeRange = (2, 4)

## Instance Generation

In [ ]:

random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)]
locations = [depot] + customers
connections = [(i, j) for i in locations for j in locations if i != j]
vehicles = [*range(1, maxNumVehicles + 1)]

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]
# dictionary of Euclidean distance for each connection (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for (i, j) in connections
}
maximalCosts = math.ceil(999 * math.sqrt(2))
# dictionary of travel times for each connection (related to the costs, scaled to time horizon)
travelTimes = {
    (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
    for (i, j) in connections
}

# create random demands, service times, and time window widths in the given range
demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
demands[0] = 0  # depot has no demand
serviceTimes = {
    i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
}
serviceTimes[0] = 0  # depot has no service time
timeWindowWidths = {
    i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
    for i in customers
}
# vehicles are allowed to leave the depot any time within the time horizon
timeWindowWidths[0] = timeHorizon

# create time windows randomly based on the previously generated information
# such that the service at a customer can be finished within the time horizon
timeWindows = {}
timeWindows[0] = (0, 0 + timeWindowWidths[0])
for i in customers:
    start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
    timeWindows[i] = (start, start + timeWindowWidths[i])

## Basic Route Model

First, we state a basic route model that uses binary variables $x_{ij}$ for each potential directed connection from location (depot or customer) $i$ to location $j$:
* The objective is to minimize the sum of costs of all used connections. 
* Each customer is visited exactly once, i.e., there is exactly one incoming connection and exactly one nextCustomer connection.
* At most $|K|$ vehicles can be used.
* We can also compute a lower bound on the number of needed vehicles, that is the sum of all demands divided by the vehicle capacity. This constraint is not necessary but sometimes helps to improve performance. Basically, this is the capacity cut for set $S = C$, see below.

\begin{align*}
\min \sum_{i,j \in L} c_{ij} x_{ij} & & \\
\sum_{i \in L} x_{ij} & = 1 & \forall j \in C \\
\sum_{j \in L} x_{ij} & = 1 & \forall i \in C \\
\sum_{j \in C} x_{0j} & \le |K| \\
\sum_{j \in C} x_{0j} & \ge \left\lceil \frac{\sum_{i \in C} d_i}{Q} \right\rceil
\end{align*}

In [ ]:
import gurobipy as gp
from gurobipy import GRB


def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)


# create model for Capacitated Vehicle Routing Problem instance
model = gp.Model("VRPTW")

# binary variables x(i,j): is 1 if some vehicle is going from node i to node j, 0 otherwise
x = model.addVars(connections, vtype=GRB.BINARY, name="x")

# objective function: minimize sum of connection costs
model.setObjective(x.prod(costs), GRB.MINIMIZE)

# all customers have exactly one incoming and one outgoing connection
model.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

# vehicle limits
model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")
model.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)


## Load and Time Models

Up to now we still have several issues in our model:
1. The vehicle capacity is ignored, so solutions might include routes with total customer demand that exceed the capacity.
2. All information related to time is ignored, so solutions might violate the time windows.
3. Less obvious is the fact that in the base route model solutions might have sub-tours, i.e., routes that are disconnected from the depot and include only customers.

Restrictions on the two resources load and time are modeled separately and linked to the route variables $x$. This allows to combine different modeling approaches for different resources. In the following we discuss some common modeling concepts that work not only for load and time but also for other changing resources along a vehicle's route, e.g., distance, number of visited customers, etc.

### Big-M Model

The following modeling approach is used very frequently since it is quite intuitive and also used in many other optimization models. 
The main idea is to associate a continuous variable to each customer that represents the state of some resource immediately before or after handling the customer. These states are then related to incoming and outgoing connections from and to other customers.

Although this approach only increases the model size slightly, the corresponding dual bounds obtained by solving the continuous relaxation are often weak. This can lead to large branch-and-bound trees and poor solution performance.

#### Load

We introduce variables $y_i \in [0,Q]$ for each location $i \in L$ to denote the vehicle load after picking up the demand at $i$.
Note that it does not matter whether we pick up demands or deliver them, as long as we stay consistent for all customers the set of feasible solutions in terms of routes stays the same.

The following constraints model the fact that if connection $(i,j)$ is used then the vehicle load at $j$ is equal to the vehicle load at $i$ plus the demand at $j$. This if-then statement can be implemented with indicator constraints or directly with a Big-M approach:
\begin{align*}
y_0 & = 0 & \\
y_i + d_j & \le y_j + Q (1 - x_{ij}) & \forall i \in L,~ j \in C,~ i \neq j \\
y_i + d_j & \ge y_j - M_{ij} (1 - x_{ij}) & \forall i \in L,~ j \in C,~ i \neq j 
\end{align*}
Since we limit the load variables by the vehicle capacity, the latter cannot be exceeded.
Additionally, sub-tours are eliminated since the load has to monotonically increase along a route. In case of a sub-tour this would result in a conflict of the load values at some point. Contrary to this, in feasible routes the increase of load values is not required when coming back to the depot (note that the constraints above are not defined for $j = 0$).
Coefficient $M_{ij}$ should be as small as possible. Here, we can set it to $M_{ij} = Q - d_i - d_j$.

#### Time

We introduce variables $z_i \in [a_i,b_i]$ for each location $i \in L$ to denote the start time of the service at $i$ that has to be within its time window.

The following constraints model the fact that if connection $(i,j)$ is used then the start time at $j$ is greater or equal to the start time at $i$ plus the service time at $i$ plus the travel time from $i$ to $j$.
\begin{align*}
z_0 & = 0 & \\
z_i + s_i + t_{ij} & \le z_j + T_{ij} (1 - x_{ij}) & \forall i \in L,~ j \in C,~ i \neq j
\end{align*}
Note that $z_j$ can be strictly greater than the left-hand side in a feasible solution if the time window start at $j$ is later.
Similar to the load model, sub-tours are eliminated since time has to monotonically increase along a route. In case of a sub-tour this would result in a conflict of the start time values at some point.
We can set coefficient $T_{ij} = b_i + s_i + t_{ij} - a_j$.


### Flow Model

Now, we extend the meaning of the variables used in the Big-M model. We do not associate resource states to customers anymore but instead to connections between customers. Those variables give us more information, not only about the resource consumption but also about the sequence of customer visits.

TODO: this is not necessarily true anymore because of time window bounds?
This modeling approach adds more variables than the Big-M model (quadratic in the number of customers). However, in most cases it pays off since it provides dual bounds at least as good as the ones of the Big-M model, and mostly much better.

#### Load

We introduce continuous variables $y_{ij} \in [0,Q]$ for each connection $(i,j)$ to denote the vehicle load after picking up the demand at $i$ and proceeding to location $j$. We can also express the new variables with the previous ones in a non-linear way, i.e., $y_{ij} = y_i \cdot x_{ij}$. In a different interpretation, these variables model the flow of goods on connection $(i,j)$ which suggests to use flow conservation constraints:
\begin{align*}
y_{0j} & = 0 & \forall j \in C \\
\sum_{i \in L} y_{ij} + d_j & = \sum_{i \in L} y_{ji} & \forall j \in C \\
y_{ij} & \ge d_i x_{ij} & \forall i \in C,~ j \in L,~ i \neq j \\
y_{ij} & \le (Q - d_j) x_{ij} & \forall i \in C,~ j \in L,~ i \neq j 
\end{align*}
The last two sets of constraints state that there can only be non-zero flow on connections that are actually chosen in the solution.
The vehicle capacity is satisfied by definition, and with a similar argument as for the Big-M model sub-tours cannot appear in solutions since the flow conservation constraints (that are not defined for $j = 0$) do not allow a route including only customers.

#### Time

We introduce variables $z_{ij} \in [0,b_i]$ for each connection $(i,j)$ to denote the start time of the service at $i$ (which must be within its time window) when immediately proceeding to location $j$. Note that the (non-linear) relation to the Big-M model variables is $z_{ij} = z_i \cdot x_{ij}$.
Then, we can define the following flow system:
\begin{align*}
z_{0j} & = 0 & \forall j \in C \\
\sum_{i \in L} \left[ z_{ij} + (s_i + t_{ij}) x_{ij} \right] & \le \sum_{i \in L} z_{ji} & \forall j \in C \\
z_{ij} & \ge a_i x_{ij} & \forall i,j \in L,~i \neq j \\
z_{ij} & \le b_i x_{ij} & \forall i,j \in L,~i \neq j
\end{align*}
Note that the flow conservation may not hold with equality if the time window start at $j$ is later.


In the following implementation, you can choose which models you want to use. This allows us to do some performance comparisons.

In [ ]:
### MODEL CONFIGURATION
loadModelType = 1  # 1: big-M, 2: flow
timeModelType = 1  # 1: big-M, 2: flow
###

## Implementation of Load and Time Models

In [ ]:
def addLoadConstraintsByBigM():

    y = model.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
    y[0].UB = 0  # empty load at depot

    model.addConstrs(
        (
            y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM1",
    )
    model.addConstrs(
        (
            y[i] + demands[j]
            >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM2",
    )


def addLoadConstraintsByFlows():

    z = model.addVars(connections, lb=0, ub=vehicleCapacity, name="z")

    for i in customers:
        z[0, i].UB = 0

    model.addConstrs(
        (z.sum("*", j) + demands[j] == z.sum(j, "*") for j in customers),
        name="flowConservation",
    )
    model.addConstrs(
        (
            z[i, j] >= demands[i] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadLowerBound",
    )
    model.addConstrs(
        (
            z[i, j] <= (vehicleCapacity - demands[j]) * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadUpperBound",
    )


def addTimeConstraintsByBigM():

    y = model.addVars(locations, name="y")
    for i in locations:
        y[i].LB = timeWindows[i][0]
        y[i].UB = timeWindows[i][1]

    model.addConstrs(
        (
            y[i] + serviceTimes[i] + travelTimes[i, j]
            <= y[j]
            + (
                timeWindows[i][1]
                + serviceTimes[i]
                + travelTimes[i, j]
                - timeWindows[j][0]
            )
            * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="timeBigM",
    )


def addTimeConstraintsByFlows():

    z = model.addVars(connections, lb=0, name="z")

    for (i, j) in connections:
        z[i, j].UB = timeWindows[i][1]

    model.addConstrs(
        (
            gp.quicksum(
                z[i, j] + (serviceTimes[i] + travelTimes[i, j]) * x[i, j]
                for i in locations
                if (i, j) in connections
            )
            <= z.sum(j, "*")
            for j in customers
        ),
        name="flowConservation",
    )
    model.addConstrs(
        (
            z[i, j] >= timeWindows[i][0] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="timeWindowStart",
    )
    model.addConstrs(
        (
            z[i, j] <= timeWindows[i][1] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="timeWindowEnd",
    )


if loadModelType == 1:
    addLoadConstraintsByBigM()
elif loadModelType == 2:
    addLoadConstraintsByFlows()

if timeModelType == 1:
    addTimeConstraintsByBigM()
elif timeModelType == 2:
    addTimeConstraintsByFlows()


## Solve Model

In [ ]:
model.params.Threads = 4
model.optimize()

## Solution Retrieval

What we basically want to know is the ordered sequence of customers each vehicle should visit.
We did not use variables that assign connections or customers to vehicles (when using the basic route model without vehicle-index), so the question is whether it is possible to retrieve a unique set of routes only based on the set of used connections defined by the $x$-variables.
This is indeed possible since each customer is visited exactly once. We iteratively pick one of the connections leaving the depot, and follow the path until we get back to the depot. Note that there is exactly one incoming and one outgoing connection at each customer, so it is clear where a route continues.

Additional to reconstructing the routes, we check whether the vehicle load does not exceed the capacity on each route and each customer is visited from the depot within its time window. This is not necessary as long as the model is correct but it helps significantly in the debugging phase.

In [ ]:
if model.SolCount >= 1:

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        time = travelTimes[0, firstCustomer]
        violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}, T:{time}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            time = max(time, timeWindows[currentCustomer][0])
            if time > timeWindows[currentCustomer][1]:
                violatedTimeWindows = True
            time += (
                serviceTimes[currentCustomer]
                + travelTimes[currentCustomer, nextCustomer[currentCustomer]]
            )
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity}, T:{time})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        if violatedTimeWindows:
            print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")

In [ ]:
if model.Status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write("iis.ilp")

In [ ]:
# free resources
model.dispose()
gp.disposeDefaultEnv()